## Import libraries

In [3]:
import requests
import datetime
import json
import smtplib
import time
import pandas
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from datetime import datetime, timedelta

## Get Your state ID
#### Now first extract your state ID

In [5]:
URL = "https://cdn-api.co-vin.in/api/v2/admin/location/states"
response = requests.get(URL)
resp_json = response.json()
pandas.DataFrame(resp_json['states'])

,state_id,state_name
0,1,Andaman and Nicobar Islands
1,2,Andhra Pradesh
2,3,Arunachal Pradesh
3,4,Assam
4,5,Bihar
5,6,Chandigarh
6,7,Chhattisgarh
7,8,Dadra and Nagar Haveli
8,37,Daman and Diu
9,9,Delhi


## Get District
#### Now find the district from the state. I am searching for Ajmer in Rajasthan.

In [7]:
URL = "https://cdn-api.co-vin.in/api/v2/admin/location/districts/29"
response = requests.get(URL)
resp_json = response.json()
pandas.DataFrame(resp_json['districts'])

,district_id,district_name
0,507,Ajmer
1,512,Alwar
2,519,Banswara
3,516,Baran
4,528,Barmer
5,508,Bharatpur
6,523,Bhilwara
7,501,Bikaner
8,514,Bundi
9,521,Chittorgarh


#### Now you can use this district id

## Variables

In [46]:
email_sender_account = "" #<ADD YOUR ACCOUNT>
email_sender_username = "" #<ADD YOUR USERNAME>. For gmail user username and account are email id.
email_sender_password = "" #<ADD YOUR PASSWORD>
email_smtp_server = "smtp.gmail.com" #In case of gmail it is smtp.gmail.com"
email_smtp_port = 587 #In case of smtp.gmail.com it is 587
email_receiver = "" #receiver email id who will get email notification

district_id = 505 #Update from the code above
next_days_to_check = 2 #It will check for next two days of availability
min_age_limit =18 #You can change it to 45 if needed. 
time_to_check_in_seconds = 10 #Latest gap to check again in second. Not recommended less than 5.
time_to_pause_after_sending_email_in_seconds = 600 #Pause for 10 mins after sending email

#protip - if login is showing less secure app, you can update it here - https://www.google.com/settings/security/lesssecureapps

In [47]:
def sendMail(recipient,email_subject,email_body):

    #login to email server
    server = smtplib.SMTP(email_smtp_server,email_smtp_port)
    server.starttls()
    server.login(email_sender_username, email_sender_password)

    print(f"Sending email to {recipient}")
    message = MIMEMultipart('alternative')
    message['From'] = email_sender_account
    message['To'] = recipient
    message['Subject'] = email_subject
    message.attach(MIMEText(email_body, 'html'))
    text = message.as_string()
    server.sendmail(email_sender_account,recipient,text)
    
    #email sent, log out.
    server.quit()

In [48]:
def fetchVaccineSlots():
    while(True):
        presentday = datetime.now()
        currentDelta = 0
        sendMailer = False
        messageList = []
        while(currentDelta<=next_days_to_check):
            
            day = presentday + timedelta(currentDelta)
            dt=day.strftime('%d-%m-%Y')
            URL = "https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id="+str(district_id)+"&date="+dt
            print(URL)
            response = requests.get(URL)
            resp_json = response.json()
            centers = resp_json['centers']
            title = "Vaccine available!!"
            for item in centers:
                session = item['sessions']
                for sess in session:
                    if (sess['available_capacity'] > 0 and sess['min_age_limit'] == min_age_limit):
                        message = "Total "+ str(sess['available_capacity']) + " vaccine is available at " + item['name'] +" with minimum age limit = "+str(sess['min_age_limit'])+" years on "+dt
                        messageList.append(message)
                        sendMailer = True
            print('No Vaccine available on '+dt)
            currentDelta+=1
                        #print(message)
        if sendMailer and len(messageList)>0:
            joined_string = "\n".join(messageList)
            print(joined_string)
            joined_string=joined_string.replace("\n","<br />\n")
            #sendMail(receiver,title,joined_string)
            #time.sleep(time_to_pause_after_sending_email_in_seconds)
            sendMailer = False

        time.sleep(time_to_check_in_seconds)

In [ ]:
fetchVaccineSlots()

https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=04-05-2021
No Vaccine available on 04-05-2021
https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=05-05-2021
No Vaccine available on 05-05-2021
https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=06-05-2021
No Vaccine available on 06-05-2021
https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=04-05-2021
No Vaccine available on 04-05-2021
https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=05-05-2021
No Vaccine available on 05-05-2021
https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=06-05-2021
No Vaccine available on 06-05-2021
https://cdn-api.co-vin.in/api/v2/appointment/sessions/public/calendarByDistrict?district_id=505&date=04-05-2021
No Vac